# Library Import

In [1]:
# !pip install wandb --upgrade
# !pip install adamp

In [2]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import cv2
import os

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
# faster rcnn model이 포함된 library
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm

import wandb

from map_boxes import mean_average_precision_for_boxes

from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from adamp import AdamP

from CosineAnnealingWarmUpRestarts import CosineAnnealingWarmUpRestarts

from sklearn.model_selection import StratifiedGroupKFold 
import json

import gc


In [3]:
# 실행 후 url을 클릭하면 API key가 나오는데 복붙하시면 됩니다!!
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jaeook (use `wandb login --relogin` to force relogin)


True

# Dataset 생성

In [4]:
class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir, transforms=None):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        
        image_id = self.coco.getImgIds(imgIds=index)  # return image id list

        image_info = self.coco.loadImgs(image_id)[0]  # return image_info
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])  # return annotation id, imgIds가 x인 annotation id return
        anns = self.coco.loadAnns(ann_ids)  # return annotation information list (annotation_info_list)

        boxes = np.array([x['bbox'] for x in anns])

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        # torchvision faster_rcnn은 label=0을 background로 취급
        # class_id를 1~10으로 수정 
        labels = np.array([x['category_id']+1 for x in anns]) 
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        areas = np.array([x['area'] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)
                                
        is_crowds = np.array([x['iscrowd'] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)

        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([index]), 'area': areas,
                  'iscrowd': is_crowds}

        # transform
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = torch.tensor(sample['bboxes'], dtype=torch.float32)

        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [5]:
class TestDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)

    def __getitem__(self, index: int):
        
        image_id = self.coco.getImgIds(imgIds=index)

        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        image = torch.tensor(image, dtype=torch.float32).permute(2,0,1)

        return image
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [6]:
def get_train_transform():
    return A.Compose([
        A.Resize(1024, 1024),
        A.OneOf([
            A.RandomBrightness(p=1),
            A.Blur(p=1),
            A.RandomFog(p=1)
        ],p=1),
        A.OneOf([
            A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1),
            A.RandomRotate90(p=1)
        ],p=1),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

# Util Functions

In [7]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


def collate_fn(batch):
    return tuple(zip(*batch))

# Trainer

In [8]:
best_loss = 1000

def train_fn(num_epochs, train_data_loader, valid_data_loader, optimizer, model, device, name, scheduler):
    #####----------------------------------------------------------------####
    # wandb.watch(model)    
    #####----------------------------------------------------------------####
    global best_loss

    train_loss_hist = Averager()
    val_loss_hist = Averager()
    
    for epoch in range(num_epochs):
        train_loss_hist.reset()
        # model.train()

        # gc.collect()
        # torch.cuda.empty_cache()

        print("Calculating training results....")
        for images, targets, image_ids in tqdm(train_data_loader):
            
            # gpu 계산을 위해 image.to(device)
            images = list(image.float().to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # calculate loss
            loss_dict = model(images, targets)

            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()

            train_loss_hist.send(loss_value)

            # backward
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

        scheduler.step()

        with torch.no_grad():
            print("Calculating validation results....")
            # model.eval()
            val_loss_hist.reset()

            for images, targets, image_ids in tqdm(valid_data_loader):
                images = list(image.float().to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                val_loss_dict = model(images, targets)

                val_losses = sum(loss for loss in val_loss_dict.values())
                val_loss_value = val_losses.item()

                val_loss_hist.send(val_loss_value)

        print(f"Epoch #{epoch+1} train_loss: {train_loss_hist.value} || val_loss: {val_loss_hist.value}")
        if val_loss_hist.value < best_loss:
            save_path = './checkpoints/faster_rcnn_torchvision_checkpoints_'+name+'.pth'
            save_dir = os.path.dirname(save_path)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            
            torch.save(model.state_dict(), save_path)
            best_loss = val_loss_hist.value

        
        #####----------------------------------------------------------------####
        wandb.log(
            {
                "train_loss": train_loss_hist.value,
                "valid_loss": val_loss_hist.value
                }
                )
        #####----------------------------------------------------------------####

In [9]:
# def train_fn(num_epochs, train_data_loader, optimizer, model, device, name, scheduler):
#     #####----------------------------------------------------------------####
#     wandb.watch(model)    
#     #####----------------------------------------------------------------####
#     best_loss = 1000
#     loss_hist = Averager()
#     for epoch in range(num_epochs):
#         loss_hist.reset()

#         for images, targets, image_ids in tqdm(train_data_loader):

#             # gpu 계산을 위해 image.to(device)
#             images = list(image.float().to(device) for image in images)
#             targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

#             # calculate loss
#             loss_dict = model(images, targets)

#             losses = sum(loss for loss in loss_dict.values())
#             loss_value = losses.item()

#             loss_hist.send(loss_value)

#             # backward
#             optimizer.zero_grad()
#             losses.backward()
#             optimizer.step()

#         scheduler.step()

        
#         print(f"Epoch #{epoch+1} loss: {loss_hist.value}")
#         if loss_hist.value < best_loss:
#             save_path = './checkpoints/faster_rcnn_torchvision_checkpoints_'+name+'.pth'
#             save_dir = os.path.dirname(save_path)
#             if not os.path.exists(save_dir):
#                 os.makedirs(save_dir)
            
#             torch.save(model.state_dict(), save_path)
#             best_loss = loss_hist.value
#         #####----------------------------------------------------------------####
#         wandb.log({"train_loss": loss_hist.value}, step=epoch)
#         #####----------------------------------------------------------------####

# evaluate function

In [10]:
def evaluate_fn(test_data_loader, model, device):
    outputs = []
    
    for images in tqdm(test_data_loader):
        # gpu 계산을 위해 image.to(device)
        images = list(image.to(device) for image in images)
        # print(len(images))  # 8
        output = model(images)
        # print(len(output))  # 8
        for img,out in zip(images,output):
            # print(len(out['boxes']))
            all_boxes=[]
            outputs.append({'boxes': out['boxes'].tolist(), 'scores': out['scores'].tolist(), 'labels': out['labels'].tolist()})
    

    return outputs

# Main

In [11]:
# 임시로 만든 config라서 뭘 추가 하면 좋을 지 같이 얘기해보면 좋을 것 같습니다!!
# config 안에 들어간 값들은 다 Wandb에 올라가서, 중요한 파라미터 들은 다 넣어야 할 것 같아요.
config = {
    'epochs': 12,
    'batch_size':32,
    'shuffle':False,
    'num_workers': 0,
    'lr': 0.005,
    'weight_decay':0.0005,
    'lr_decay_step':2,
    'momentum':0.9,
    'score_threshold': 0.05,
    "augmentation": ["RandomRotate90(p=0.5)", "RandomBrightness(p=0.5)"],
    "scheduler": 'CosineAnnealingWarmUpRestarts(T_0=6, T_mult=1, eta_max=0.02, T_up=2, gamma=0.5 )',
    "optimizer": 'sgd'
}


In [12]:
# annotation = '../../dataset/train.json' # ann
# coco = COCO(annotation)



In [13]:
def main(config=None):
    name = "kfold_augmentation_2"
    #####--------------------------------wandb 연결 및 config 지정-------------------------------------------------####
    # project : 'project 이름' ,
    # entity : '팀 이름'
    wandb.init(project='object_detection', entity='cv-3-bitcoin', config = config, reinit=True)
    wandb.run.name = "jaeook_test_"+name  # 실험의 이름 지정
    config = wandb.config
    #####---------------------------------------------------------------------------------------------####
    
    # 데이터셋 불러오기
    annotation = '../../dataset/train.json' # annotation 경로
    data_dir = '../../dataset' # data_dir 경로

    #####-------------------------------------------------#####
    # with open(annotation) as f: data = json.load(f)
    coco = COCO(annotation)
    data = coco.loadAnns(list(range(0,23144)))
    var = [(ann['image_id'], ann['category_id']) for ann in data]
    # var = [(ann['image_id'], ann['category_id']) for ann in data['annotations']]
    # X = np.ones((len(data['annotations']),1))   # (bbox 개수, 1)
    X = np.ones((len(data),1))   # (bbox 개수, 1)
    y = np.array([v[1] for v in var])   # category_id
    groups = np.array([v[0] for v in var])  # image_id


    cv = StratifiedGroupKFold(n_splits=5)


    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(device)
    
    # torchvision model 불러오기
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 11 # class 개수= 10 + background

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model.to(device)
    params = [p for p in model.parameters() if p.requires_grad]
    
    optimizer = torch.optim.SGD(params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay)
    # optimizer = torch.optim.AdamW(params, lr=config.lr, weight_decay = config.weight_decay)
    num_epochs = config.epochs
    # scheduler = StepLR(optimizer, config.lr_decay_step, gamma=0.5)
    # scheduler = CosineAnnealingLR(optimizer, T_max=6, eta_min=0)
    scheduler = CosineAnnealingWarmUpRestarts(optimizer,T_0=6, T_mult=1, eta_max=0.02, T_up=2, gamma=0.5 )
    # T_0 : 최초 주기값,
    # T_mult는 주기가 반복되면서 최초 주기값에 비해 얼만큼 주기를 늘려나갈 것인지 스케일 값
    # eta_max는 learning rate의 최댓값
    # T_up은 Warm up 시 필요한 epoch 수를 지정하며 일반적으로 짧은 epoch 수를 지정
    # gamma는 주기가 반복될수록 eta_max 곱해지는 스케일값

    # scheduler = ReduceLROnPlateau(
    #     optimizer,
    #     factor=0.5,   # 학습률이 감소하는 요인입니다. new_lr = lr * factor.
    #     patience=config.lr_decay_step,
    #     threshold=0.001,  
    #     verbose=True,  
    #     min_lr=1e-4,  
    #     threshold_mode="abs",
    # )
    wandb.watch(model, log='all', log_freq=2)    

    n = 0
#---------------------------------------추가--------------------------------------------------------#
    

#---------------------------------------------KFold-------------------------------------------------------#

    for train_idx, val_idx in cv.split(X,y,groups):

        train_idx = list(set(groups[train_idx])) # 중복되는 index 제거
        val_idx = list(set(groups[val_idx]))

        train_dataset = torch.utils.data.Subset(CustomDataset(annotation, data_dir, get_train_transform()), indices=train_idx)
        train_data_loader = DataLoader(
            train_dataset,
            batch_size=config.batch_size,
            shuffle=config.shuffle,
            num_workers=config.num_workers,
            collate_fn=collate_fn
        )

        valid_dataset = torch.utils.data.Subset(CustomDataset(annotation, data_dir, get_valid_transform()), indices=val_idx)
        valid_data_loader = DataLoader(
            valid_dataset,
            batch_size=config.batch_size,
            shuffle=config.shuffle,
            num_workers=config.num_workers,
            collate_fn=collate_fn
        )

        n += 1

        print("="*50)
        print(f"Fold : {n}")        

        
        # training
        train_fn(num_epochs, train_data_loader,valid_data_loader, optimizer, model, device, name, scheduler)


    #####--------------------------------calculate mAP------------------------------------------------------####

    check_point = './checkpoints/faster_rcnn_torchvision_checkpoints_'+ name + '.pth'
    
    # get number of input features for the classifier
    model_eval = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model_eval.roi_heads.box_predictor.cls_score.in_features
    model_eval.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model_eval.to(device)        
    model_eval.load_state_dict(torch.load(check_point))
    model_eval.eval()

    valid_dataset = TestDataset(annotation, data_dir)
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size = 16,
        shuffle = False,
        num_workers = 4
    )

    # predict
    outputs = evaluate_fn(valid_data_loader, model_eval, device)
    prediction_strings = []
    file_names = []
    
    # coco = COCO(annotation)

    # submission 파일 생성
    for i, output in enumerate(outputs):
        prediction_string = ''
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
            if score > config.score_threshold: 
                # label[1~10] -> label[0~9]
                prediction_string += str(label-1) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
                    box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
        prediction_strings.append(prediction_string)
        file_names.append(image_info['file_name'])
    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = file_names
    submission.to_csv('./faster_rcnn_torchvision_train_submission.csv', index=None)
    

    PRED_CSV = './faster_rcnn_torchvision_train_submission.csv'
    LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal", 
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

    pred_df = pd.read_csv(PRED_CSV)

    new_pred = []

    file_names = pred_df['image_id'].values.tolist()
    bboxes = pred_df['PredictionString'].values.tolist()
    
    for i, bbox in enumerate(bboxes):
        if isinstance(bbox, float):
            print(f'{file_names[i]} empty box')

    for file_name, bbox in tqdm(zip(file_names, bboxes)):
        boxes = np.array(str(bbox).split(' '))
    
        if len(boxes) % 6 == 1:
            boxes = boxes[:-1].reshape(-1, 6)
        elif len(boxes) % 6 == 0:
            boxes = boxes.reshape(-1, 6)
        else:
            raise Exception('error', 'invalid box count')
        for box in boxes:
            new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])


    gt = []

    for image_id in coco.getImgIds():  # GT의 image_id
        
        image_info = coco.loadImgs(image_id)[0]    
        annotation_id = coco.getAnnIds(imgIds=image_info['id'])
        annotation_info_list = coco.loadAnns(annotation_id)  # return annotation information list
        
        file_name = image_info['file_name']
        
        for annotation in annotation_info_list:
            gt.append([file_name, annotation['category_id'],
                    float(annotation['bbox'][0]),
                    float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                    float(annotation['bbox'][1]),
                    (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

    mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5)

    wandb.log({"mAP": mean_ap})
    wandb.run.save()
    #####---------------------------------------------------------------------------------------------####

In [14]:
# def main(config=None):
#     name = "CosineAnealingWarmUpRestarts_threshold0.025"
#     #####--------------------------------wandb 연결 및 config 지정-------------------------------------------------####
#     # project : 'project 이름' ,
#     # entity : '팀 이름'
#     wandb.init(project='object_detection', entity='cv-3-bitcoin', config = config, reinit=True)
#     wandb.run.name = "jaeook_test_"+name  # 실험의 이름 지정
#     config = wandb.config
#     #####---------------------------------------------------------------------------------------------####
    
#     # 데이터셋 불러오기
#     annotation = '../../dataset/train.json' # annotation 경로
#     data_dir = '../../dataset' # data_dir 경로


#     train_dataset = CustomDataset(annotation, data_dir, get_train_transform()) 
#     train_data_loader = DataLoader(
#         train_dataset,
#         batch_size=config.batch_size,
#         shuffle=config.shuffle,
#         num_workers=config.num_workers,
#         collate_fn=collate_fn
#     )
#     device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#     print(device)
    
#     # torchvision model 불러오기
#     model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
#     num_classes = 11 # class 개수= 10 + background

#     # get number of input features for the classifier
#     in_features = model.roi_heads.box_predictor.cls_score.in_features
#     model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
#     model.to(device)
#     params = [p for p in model.parameters() if p.requires_grad]
    
#     optimizer = torch.optim.SGD(params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay)
#     # optimizer = torch.optim.AdamW(params, lr=config.lr, weight_decay = config.weight_decay)
#     num_epochs = config.epochs
#     # scheduler = StepLR(optimizer, config.lr_decay_step, gamma=0.5)
#     # scheduler = CosineAnnealingLR(optimizer, T_max=6, eta_min=0)
#     scheduler = CosineAnnealingWarmUpRestarts(optimizer,T_0=6, T_mult=1, eta_max=0.02, T_up=2, gamma=0.5 )
#     # T_0 : 최초 주기값,
#     # T_mult는 주기가 반복되면서 최초 주기값에 비해 얼만큼 주기를 늘려나갈 것인지 스케일 값
#     # eta_max는 learning rate의 최댓값
#     # T_up은 Warm up 시 필요한 epoch 수를 지정하며 일반적으로 짧은 epoch 수를 지정
#     # gamma는 주기가 반복될수록 eta_max 곱해지는 스케일값

#     # scheduler = ReduceLROnPlateau(
#     #     optimizer,
#     #     factor=0.5,   # 학습률이 감소하는 요인입니다. new_lr = lr * factor.
#     #     patience=config.lr_decay_step,
#     #     threshold=0.001,  
#     #     verbose=True,  
#     #     min_lr=1e-4,  
#     #     threshold_mode="abs",
#     # )
    
#     # training
#     train_fn(num_epochs, train_data_loader, optimizer, model, device, name, scheduler)


#     #####--------------------------------calculate mAP------------------------------------------------------####
#     valid_dataset = ValidDataset(annotation, data_dir)
#     valid_data_loader = DataLoader(
#         valid_dataset,
#         batch_size=config.batch_size,
#         shuffle=config.shuffle,
#         num_workers=config.num_workers
#     )

#     score_threshold = 0.05

#     model.eval()

#     # predict
#     outputs = evaluate_fn(valid_data_loader, model, device)
#     prediction_strings = []
#     file_names = []
    
#     coco = COCO(annotation)

#     # submission 파일 생성
#     for i, output in enumerate(outputs):
#         prediction_string = ''
#         image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
#         for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
#             if score > config.score_threshold: 
#                 # label[1~10] -> label[0~9]
#                 prediction_string += str(label-1) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
#                     box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
#         prediction_strings.append(prediction_string)
#         file_names.append(image_info['file_name'])
#     submission = pd.DataFrame()
#     submission['PredictionString'] = prediction_strings
#     submission['image_id'] = file_names
#     submission.to_csv('./faster_rcnn_torchvision_train_submission.csv', index=None)
    

#     PRED_CSV = './faster_rcnn_torchvision_train_submission.csv'
#     LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal", 
#               "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

#     pred_df = pd.read_csv(PRED_CSV)

#     new_pred = []

#     file_names = pred_df['image_id'].values.tolist()
#     bboxes = pred_df['PredictionString'].values.tolist()
    
#     for i, bbox in enumerate(bboxes):
#         if isinstance(bbox, float):
#             print(f'{file_names[i]} empty box')

#     for file_name, bbox in tqdm(zip(file_names, bboxes)):
#         boxes = np.array(str(bbox).split(' '))
    
#         if len(boxes) % 6 == 1:
#             boxes = boxes[:-1].reshape(-1, 6)
#         elif len(boxes) % 6 == 0:
#             boxes = boxes.reshape(-1, 6)
#         else:
#             raise Exception('error', 'invalid box count')
#         for box in boxes:
#             new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])


#     gt = []

#     for image_id in coco.getImgIds():  # GT의 image_id
        
#         image_info = coco.loadImgs(image_id)[0]    
#         annotation_id = coco.getAnnIds(imgIds=image_info['id'])
#         annotation_info_list = coco.loadAnns(annotation_id)  # return annotation information list
        
#         file_name = image_info['file_name']
        
#         for annotation in annotation_info_list:
#             gt.append([file_name, annotation['category_id'],
#                     float(annotation['bbox'][0]),
#                     float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
#                     float(annotation['bbox'][1]),
#                     (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

#     mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5)

#     wandb.log({"mAP": mean_ap})
#     wandb.run.save()
#     #####---------------------------------------------------------------------------------------------####

In [15]:
if __name__ == '__main__':
    main(config)

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
cuda
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
loading annotations into memory...
Done (t=0.24s)
creating index...
index created!
Fold : 1
Calculating training results....


100%|██████████| 123/123 [11:49<00:00,  5.77s/it]


Calculating validation results....


100%|██████████| 31/31 [00:56<00:00,  1.81s/it]


Epoch #1 train_loss: 0.8139808323325181 || val_loss: 0.62773383432819
Calculating training results....


...

Epoch #15 train_loss: 0.34879107494664385 || val_loss: 0.3585759420548716
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


100%|██████████| 306/306 [02:58<00:00,  1.72it/s]
4883it [00:01, 4366.44it/s]


Number of files in annotations: 4883
Number of files in predictions: 4883
Unique classes: 10
Detections length: 4883
Annotations length: 4883
0                              | 0.478237 |    3966
1                              | 0.607693 |    6352
2                              | 0.710326 |     897
3                              | 0.664225 |     936
4                              | 0.775143 |     982
5                              | 0.583886 |    2943
6                              | 0.640959 |    1263


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


7                              | 0.788395 |    5178
8                              | 0.879740 |     159
9                              | 0.739063 |     468
mAP: 0.686767
